### 1. Instalação das bibliotecas necessárias

In [1]:
%pip install requests pandas plyer

Note: you may need to restart the kernel to use updated packages.


### 2. Importação das bibliotecas

In [2]:
import requests
import pandas as pd
import sqlite3
from plyer import notification
from datetime import datetime

### 3. Def de criação do alerta

In [3]:
def alerta(nivel, base, etapa):
  if nivel == 1:
    alerta = 'Baixo'
  elif nivel == 2:
    alerta = 'Médio'
  else:
    alerta = 'Alto'

  title = f"Alerta {alerta}"
  message = f"Falha no carregamento da base {base} na etapa {etapa} \n{datetime.now()}"
  notification.notify(
    title=title,
    message=message,
    app_name="etl_pipeline",
    timeout=10
  )

### 4. Def para extrair dados da API

In [4]:
def extrair_dados_da_api(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        alerta(3, url, "Extração")
        print(f"ERRO: Falha ao buscar dados da API. Código de status: {response.status_code}")

### 5. Def para extrair dados detalhados dos Pokemóns

In [5]:
def extrair_todas_paginas(base_url):
    results = []
    url = base_url
    while url:
        data = extrair_dados_da_api(url)
        results.extend(data['results'])
        url = data.get('next')
    return results

### 6. Def para extrair 3 tabelas da API e relacionar dados

In [6]:
def extrair_tabelas():
    base_urls = {
        'pokemons': "https://pokeapi.co/api/v2/pokemon?limit=100",
        'habilidades': "https://pokeapi.co/api/v2/ability?limit=100",
        'tipos': "https://pokeapi.co/api/v2/type?limit=100"
    }
    
    tabelas = {}
    
    for nome, base_url in base_urls.items():
        print(f"Extraindo dados de {nome}...")
        resultados = extrair_todas_paginas(base_url)
        tabelas[nome] = pd.DataFrame(resultados)
    
    return tabelas['pokemons'], tabelas['habilidades'], tabelas['tipos']

### 7. Def para carregar os dados no banco de dados

In [7]:
def carregar_dados_para_db(df, db_name="../pokeapi_data.db", table_name="tabela"):
    conn = sqlite3.connect(db_name)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()
    print(f"Dados carregados para {db_name} na tabela {table_name}")

### 8. Execução da extração, transformação e carga dos dados

In [8]:
url = "https://pokeapi.co/api/v2"

In [9]:
# Extração dos dados
df_pokemons, df_habilidades, df_tipos = extrair_tabelas()

Extraindo dados de pokemons...
Extraindo dados de habilidades...
Extraindo dados de tipos...


In [10]:
# Carregar dados no banco de dados
carregar_dados_para_db(df_pokemons, db_name="../pokeapi_data.db", table_name="pokemons")
carregar_dados_para_db(df_habilidades, db_name="../pokeapi_data.db", table_name="habilidades")
carregar_dados_para_db(df_tipos, db_name="../pokeapi_data.db", table_name="tipos")

Dados carregados para ../pokeapi_data.db na tabela pokemons
Dados carregados para ../pokeapi_data.db na tabela habilidades
Dados carregados para ../pokeapi_data.db na tabela tipos


### 9. Visualização dos dados usando queries SQL

In [11]:
# Visualização dos dados usando queries SQL
def executar_query(query, db_name="../pokeapi_data.db"):
    conn = sqlite3.connect(db_name)
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

In [12]:
# Tabela nomes dos pokemóns
query_pokemons = "SELECT * FROM pokemons"
df_pokemons_query = executar_query(query_pokemons)
print(df_pokemons_query.head())

         name                                   url
0   bulbasaur  https://pokeapi.co/api/v2/pokemon/1/
1     ivysaur  https://pokeapi.co/api/v2/pokemon/2/
2    venusaur  https://pokeapi.co/api/v2/pokemon/3/
3  charmander  https://pokeapi.co/api/v2/pokemon/4/
4  charmeleon  https://pokeapi.co/api/v2/pokemon/5/


In [13]:
# Tabela habilidades
query_habilidades = "SELECT * FROM habilidades"
df_habilidades_query = executar_query(query_habilidades)
print(df_habilidades_query.head())

           name                                   url
0        stench  https://pokeapi.co/api/v2/ability/1/
1       drizzle  https://pokeapi.co/api/v2/ability/2/
2   speed-boost  https://pokeapi.co/api/v2/ability/3/
3  battle-armor  https://pokeapi.co/api/v2/ability/4/
4        sturdy  https://pokeapi.co/api/v2/ability/5/


In [14]:
# Tabela tipos
query_tipos = "SELECT * FROM tipos"
df_tipos_query = executar_query(query_tipos)
print(df_tipos_query.head())

       name                                url
0    normal  https://pokeapi.co/api/v2/type/1/
1  fighting  https://pokeapi.co/api/v2/type/2/
2    flying  https://pokeapi.co/api/v2/type/3/
3    poison  https://pokeapi.co/api/v2/type/4/
4    ground  https://pokeapi.co/api/v2/type/5/


### 10. Exemplo de erro na extração da base

In [15]:
extrair_dados_da_api("https://pokeapi.co/api/v2/pokemon/1/v2")

ERRO: Falha ao buscar dados da API. Código de status: 404
